# Spectral clustering
The demo for spectral clustering

In [2]:
import pandas as pd
from pylab import text
from scipy.stats import entropy
from scipy.stats import ttest_rel
from scipy import interpolate
import scipy.io as sio
import scipy.stats
import numpy as np

In [3]:
## The individual variability
df_l_opt = pd.read_csv('/Users/fiona/Library/CloudStorage/OneDrive-mail.bnu.edu.cn/Project/PAC_Individual_difference/result/stats/df_left_thr0.1_stats.csv')
df_r_opt = pd.read_csv('/Users/fiona/Library/CloudStorage/OneDrive-mail.bnu.edu.cn/Project/PAC_Individual_difference/result/stats/df_right_thr0.1_stats.csv')

## Similarity matrix

In [4]:
## left
l_variability = df_l_opt['l_variability']
n = len(l_variability)
l_sim = np.zeros((len(l_variability),len(l_variability) ))

for k, val in enumerate(l_variability):
    tmp = np.array([val]*n)
    l_sim[k, :] = abs(tmp - l_variability)
l_sim = 1 - l_sim

## right

r_variability = df_r_opt['r_variability']
n = len(r_variability)
r_sim = np.zeros((len(r_variability),len(r_variability) ))

for k, val in enumerate(r_variability):
    tmp = np.array([val]*n)
    r_sim[k, :] = abs(tmp - r_variability)
r_sim = 1 - r_sim

## Clustering

In [ ]:
from sklearn import cluster, metrics

def LJH_SpectralClustering_fit(InputMatrix,OutFile,ClassNum, rand_seed, Gamma): 

    gamma=Gamma
    max_CH_Score=0
    for k in ClassNum:
        clustering = cluster.SpectralClustering(n_clusters=k,gamma=Gamma,assign_labels="discretize",random_state = rand_seed).fit(InputMatrix)
        clustering.labels_
        score=metrics.calinski_harabasz_score(InputMatrix,clustering.labels_)

        print ("Calinski-Harabaz Score with gamma=", gamma, "n_clusters=", k,"score:", score)
        if score>max_CH_Score:
            max_CH_Score=score
            optimal_gamma=gamma
            optimal_n_cluster=k
        optimal_params=dict(gamma=optimal_gamma,n_cluster=optimal_n_cluster,CH_score = max_CH_Score)
    
    opt_clustering= cluster.SpectralClustering(n_clusters = optimal_n_cluster,gamma=optimal_gamma,assign_labels="discretize",random_state=rand_seed).fit(InputMatrix)
          
    sio.savemat(OutFile,{'Clustering_labels':opt_clustering.labels_+1,'OptParams':optimal_params}) # plus 1 for
    
    return  optimal_params,opt_clustering.labels_+1  # start from 1


## Different rand seeds (gamma = 0.001)

In [ ]:
n_class = [4]
Gamma = 0.001
rand_seeds = np.arange(0,1000,5)

for rand_seed in rand_seeds:

    out_file_l = '/Users/fiona/Library/CloudStorage/OneDrive-mail.bnu.edu.cn/Project/PAC_Individual_difference/result/stats/clustering/c_vs_hcpmmp/robust/orig_clustering/new/left/cluster_4_left' + '_' + 'rand_seed_' + str(rand_seed) + '.mat'
    paras_l, labels_l = LJH_SpectralClustering_fit(l_sim,out_file_l,n_class, rand_seed, Gamma)
    print(paras_l)
    out_file_r = '/Users/fiona/Library/CloudStorage/OneDrive-mail.bnu.edu.cn/Project/PAC_Individual_difference/result/stats/clustering/c_vs_hcpmmp/robust/orig_clustering/new/right/cluster_4_right' + '_' + 'rand_seed_' + str(rand_seed) + '.mat'
    paras_r, labels_r = LJH_SpectralClustering_fit(r_sim,out_file_r,n_class, rand_seed, Gamma)
    print(paras_r)

## View the results

In [5]:
from surfplot import Plot
from neuromaps.datasets import fetch_fslr
 
def view_clusters(lpac_data, rpac_data, save_name = None):

    ## 1. plot surf
    import matplotlib.colors as mcolors
    #my_colors = ['black', 'red', 'yellow', 'green','blue']
    #my_colors = ['blue', 'green', 'yellow', 'red','black']
    my_colors = ['blue', 'green', 'yellow','red']
    my_cmap = mcolors.ListedColormap(my_colors, name="my_cmap")
    surfaces = fetch_fslr()
    #lh, rh = surfaces['inflated']
    lh, rh = surfaces['sphere']
    lh1, rh1 = surfaces['veryinflated']

    # build
    pac = Plot(surf_lh=lh, surf_rh=rh, views = 'lateral',size = (500,250),zoom = 1.5, brightness = 0.6)
    pac1 = Plot(surf_lh=lh1, surf_rh=rh1, views = 'lateral',size = (500,200),zoom = 1.5, brightness = 0.6)
    
    pac.add_layer({'left': lpac_data, 'right': rpac_data}, cmap = my_cmap, cbar = False, zero_transparent = True, as_outline = False)
    pac1.add_layer({'left': lpac_data, 'right': rpac_data}, cmap = my_cmap, cbar = False, zero_transparent = True, as_outline = False)
    fig = pac.build()
    fig1 = pac1.build()
    #fig = pac1.build()
    #pac.colorbar(fig, extend='min', extendrect=True)
    fig.show() # bg: sphere
    fig1.show() # bg: veryinflated

    if save_name is not None:
        fig.savefig(save_name, facecolor='white', dpi=1000) # background color: white
    plt.close()

In [ ]:
## file orig-view

rand_seeds = np.arange(0,100,5) # view 20 files
Gamma = 0.001
for rand_seed in rand_seeds:

    file_l = '/Users/fiona/Library/CloudStorage/OneDrive-mail.bnu.edu.cn/Project/PAC_Individual_difference/result/stats/clustering/c_vs_hcpmmp/robust/orig_clustering/new/left/cluster_4_left_rand_seed_' + str(rand_seed) + '.mat'
    file_r = '/Users/fiona/Library/CloudStorage/OneDrive-mail.bnu.edu.cn/Project/PAC_Individual_difference/result/stats/clustering/c_vs_hcpmmp/robust/orig_clustering/new/right/cluster_4_right_rand_seed_' + str(rand_seed) + '.mat'
    left = sio.loadmat(file_l)
    right = sio.loadmat(file_r)
    labels_l = left['Clustering_labels'][0,:]
    labels_r = right['Clustering_labels'][0,:]
    vert_index_l = df_l_opt['vert_matlab'] -1 
    vert_index_r = df_r_opt['vert_matlab'] -1 
    lpac_data = np.zeros((32492,))
    rpac_data = np.zeros((32492,))
    lpac_data[vert_index_l] = labels_l
    rpac_data[vert_index_r] = labels_r
    
#     # save 32k files
#     save_32k_l = file_l.replace('orig_clustering', 'orig_clustering_32k')
#     save_32k_r = file_r.replace('orig_clustering', 'orig_clustering_32k')
    
#     sio.savemat(save_32k_l, {'lpac_data': lpac_data})
#     sio.savemat(save_32k_r, {'rpac_data': rpac_data})
    
       
    #view and save fig

    save_dir = '/Users/fiona/Library/CloudStorage/OneDrive-mail.bnu.edu.cn/Project/PAC_Individual_difference/result/stats/clustering/c_vs_hcpmmp/robust/orig_clustering/new/figs'
    save_name = save_dir + '/' + 'custers_lr_rand_seed_' + str(rand_seed) + '.png'
    view_clusters(lpac_data, rpac_data, save_name)